### Workflow

1. Start with a question
2. Get and clean the data
3. Perform EDA
4. Apply techniques
5. Share insights

###### Define the scope of your project based on your domain expertise

---

### 1. What makes Astroworld so good?
##### Need better questions...

#### Initialization

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from collections import defaultdict
import contractions
import pandas as pd
import requests
import re
import os

In [2]:
stop_words = stopwords.words('english')

options = Options()
options.headless = True

#### Get album link

artist = 'travis scott'
album = 'astroworld'

browser = Chrome(options=options)
browser.get(f'https://www.google.com/search?q=genius+album+{artist}+{album}'.replace(' ', '+'))
page = browser.page_source
browser.quit()

soup = bs(page, 'html.parser')

album_url = soup.find(class_='r').a['href']
album_url

#### Get song names + links

browser = Chrome(options=options)
browser.get(album_url)
page = browser.page_source
browser.quit()

soup = bs(page, 'html.parser')

    # numTracks = len(soup.find_all('album-tracklist-row'))

### What should I do with songs with features? 

Parse out main artist's verses or scrap in general?

#### Get lyrics

track_names = [' '.join(item.h3.text.strip().split()[:-1]) for item in soup.find_all('album-tracklist-row')]
track_urls = [item.a['href'] for item in soup.find_all('album-tracklist-row')]

all_lyrics = []

for url in track_urls:
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    
    all_lyrics.append(soup.find_all('p')[0].text)

data_dict = {
    'song': track_names,
    'url': track_urls,
    'lyrics': all_lyrics
}

    # import pickle

    # with open('datadict.pkl', 'wb') as f:
    #     pickle.dump(data_dict, f)

In [3]:
import pickle

with open('datadict.pkl', 'rb') as f:
    data_dict = pickle.load(f)

In [4]:
df1 = pd.DataFrame(data_dict)
# df1.index = [i+1 for i in df1.index]

In [5]:
def fix_song_names():
    new_index = []
    
    for song_name in df1.song:
        new_index.append(f"{song_name.split('(')[0].strip()}")
        
    return new_index

In [6]:
df1.index = fix_song_names()
df1.drop(columns=['song'], axis=1, inplace=True)

In [7]:
df1

,url,lyrics
STARGAZING,https://genius.com/Travis-scott-stargazing-lyrics,"[Part I]\n\n[Chorus]\nRollin', rollin', rollin..."
CAROUSEL,https://genius.com/Travis-scott-carousel-lyrics,[Intro: Big Tuck]\nWhat's crackin'?\nYou alrea...
SICKO MODE,https://genius.com/Travis-scott-sicko-mode-lyrics,"[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun i..."
R.I.P. SCREW,https://genius.com/Travis-scott-rip-screw-lyrics,"[Intro: Swae Lee, DJ Screw & Travis Scott]\n(Y..."
STOP TRYING TO BE GOD,https://genius.com/Travis-scott-stop-trying-to...,[Intro]\nThis is the real action: the pot part...
NO BYSTANDERS,https://genius.com/Travis-scott-no-bystanders-...,[Intro: Juice WRLD]\nThe party never ends\nIn ...
SKELETONS,https://genius.com/Travis-scott-skeletons-lyrics,"[Intro: Travis Scott, Pharrell Williams, Kevin..."
WAKE UP,https://genius.com/Travis-scott-wake-up-lyrics,[Chorus 1: The Weeknd]\nI don't wanna wake up\...
5% TINT,https://genius.com/Travis-scott-5-tint-lyrics,[Chorus]\nWho's that creeping through my windo...
NC-17,https://genius.com/Travis-scott-nc-17-lyrics,[Intro: Travis Scott]\nOoh\n\n[Chorus: Travis ...


In [8]:
# def stem(word):
#     from nltk.stem import PorterStemmer
#     ps = PorterStemmer()
    
#     return ps.stem(word)


# def lemmatize_noun(word):
#     wnl = WordNetLemmatizer()
    
#     return wnl.lemmatize(word, 'n')


def lemmatize_verb(word):
    wnl = WordNetLemmatizer()
    
    return wnl.lemmatize(word, 'v')


def replace_slang(word):
    word = word.lower()
    
    if word.endswith("in'"):
        wnl = WordNetLemmatizer()

        #og_len = len(word)

        replaced = str(word[:word.rindex('in')] + 'ing')

        replaced_lemma = lemmatize_verb(replaced)
        #replaced_lemma_len = len(replaced_lemma)

        '''words like:
            filling, drawing, blessing, working, wiring, matting, funding (https://ell.stackexchange.com/questions/121671/noun-ends-with-ing-but-not-a-gerund)
        makes this weird'''
        if wordnet.synsets(replaced_lemma): 
        #if replaced_lemma_len < og_len:
            return replaced
            #return replaced_lemma
        else:
            # funky word, check if synsets
            if wordnet.synsets(word):
                return word
            return word #+ '---NONE---'
    else:
        return word

In [9]:
# remove music tags and adlibs.. text in [] and ()
df1.lyrics = df1.lyrics.apply(lambda x: re.sub(r'\[.*\]', '', x).lower().strip())
df1.lyrics = df1.lyrics.apply(lambda x: re.sub(r'\(.*\)', '', x).strip())

In [10]:
# remove numbers
df1.lyrics = df1.lyrics.apply(lambda x: re.sub(r'\d+', ' ', x).strip())

In [11]:
# remove new lines
df1.lyrics = df1.lyrics.apply(lambda x: re.sub(r'\n', ' ', x))

In [12]:
# words with hyphens

for i in df1.lyrics.apply(lambda x: [j for j in x.split() if "-" in j]):
    print(i)
    print('==================================')

[]
['m-e-d', 'drive-thru', 'drive-thru']
['some-some-some-someone', "pl-pl-playin'", "pl-pl-playin'", "pl-playin'"]
[]
['hmm-hmm', 'hmm-hmm-hmm,', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm,', 'hmm-hm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hook-up', 'nine-to-five', 'hmm-hmm', 'hmm-hmm,', 'hmm-hm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm,', 'hmm-hm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'hmm-hmm', 'woah-oh-oh', 'woah-oh-oh', 'woah-oh-oh', 'woah-oh-oh']
[]
[]
['ice-t']
['out-your-name', 'fall-in-love', 'out-your-name', 'out-iced', 'out-your-name']
['t-shirt']
[]
['two-tone', 'mmm-hm-mmm-hm-mmm-hm-mmm', 'mmm-hm-mmm-hm-mmm-hm-mmm', 'mmm-hm-mmm-hm-mmm-hm-mmm', 'mmm-hm-mmm-hm-mmm-hm-mmm', 'mmm-hm-mmm-hm-mmm-hm-mmm', 'two-tone']
['uh-huh,', 'uh-huh,']
['f-o-i']
[]
['per-request,', 'no-no,']
['bad-bad', 'half-rinsed', 'cul-de-sacs']


In [13]:
# remove non-apostrophe punctuation
df1.lyrics = df1.lyrics.apply(lambda x: re.sub(r"[^a-zA-z']", ' ', x))

In [14]:
# contraction checking...

for i in df1.lyrics:
    print([contractions.fix(j) for j in i.split() if "'" in j])
    print('==================================')

["rollin'", "rollin'", "rollin'", "stargazin'", "sippin'", "feelin'", 'going', 'going', "rollin'", "rollin'", "rollin'", "stargazin'", 'going', "femalin'", "excellin'", "intellin'", "propellin'", 'going', 'we will', ' them', "callin'", 'it is', 'he is', "rollin'", "rollin'", "rollin'", "stargazin'", "sippin'", "feelin'", 'going', 'going', 'going', 'are not', 'that is', "'", 'I would', 'that is', "packin'", 'I am', 'are not', 'are not', ' them', "divin'", 'until', "talkin'", 'about', "b's", "jumpin'", 'I am', 'I am', 'I am', 'I am']
['what is', "crackin'", 'I am', 'are not', "takin'", 'do not', "bustin'", 'what are', "talkin'", 'what is', "pertainin'", 'I am', "invadin'", "hesitatin'", "bitin'", "bitin'", 'it is', 'that is', 'it is', 'you are', 'I am', 'do not', 'it is', "movin'", 'I am', 'that is', 'it is', 'you are', 'I am']
["freezin'", 'that is', "winter's", 'that is', 'do not', 'nothing', ' them', ' them', 'going', 'that is', "hoppin'", 'it is', 'I am', "poppin'", "sellin'", "choki

In [15]:
# handle contractions
df1.lyrics = df1.lyrics.apply(lambda x: [contractions.fix(word) for word in x.split()])
df1.lyrics = df1.lyrics.apply(lambda x: ' '.join(x))

    # not terriblly import but check for words ending with "'s"
    for i in df1.lyrics:
        #print([j.split("'") for j in i.split() if j.endswith("'s")])
        print([j for j in i.split() if j.endswith("'s")])
        print('==================================')

In [16]:
# words ending in "in'"

z = df1.lyrics.apply(lambda x: [i for i in x.split() if i not in stop_words])
y = z.apply(lambda x: [i for i in x if i[-3:] == "in'"])

for i in y:
    print(i)
    print('============================================')

["rollin'", "rollin'", "rollin'", "stargazin'", "sippin'", "feelin'", "rollin'", "rollin'", "rollin'", "stargazin'", "femalin'", "excellin'", "intellin'", "propellin'", "callin'", "rollin'", "rollin'", "rollin'", "stargazin'", "sippin'", "feelin'", "packin'", "divin'", "talkin'", "jumpin'"]
["crackin'", "takin'", "bustin'", "talkin'", "pertainin'", "invadin'", "hesitatin'", "bitin'", "bitin'", "movin'"]
["freezin'", "hoppin'", "poppin'", "sellin'", "chokin'", "playin'", "playin'", "comin'", "playin'", "spinnin'", "sendin'", "sendin'", "jumpin'", "workin'"]
["sayin'", "playin'", "beatin'"]
["comin'", "wantin'", "keepin'", "peelin'", "comin'"]
["flippin'", "grippin'", "dodgin'", "walkin'"]
[]
["flowin'", "gettin'", "creepin'", "controllin'", "dreamin'", "lookin'", "dreamin'"]
["creepin'", "bouncin'", "feelin'"]
["hangin'", "sippin'", "vibratin'", "talkin'", "thinkin'", "fuckin'", "hangin'", "sippin'", "vibratin'"]
["wildin'"]
["drippin'"]
["smokin'", "smokin'", "callin'", "smokin'"]
["sa

In [17]:
# replace "in'" with 'ing'

x = y.apply(lambda x: [replace_slang(i) for i in x])

for i in x:
    print(i)
    print('==========================================')

['rolling', 'rolling', 'rolling', 'stargazing', 'sipping', 'feeling', 'rolling', 'rolling', 'rolling', 'stargazing', "femalin'", 'excelling', "intellin'", 'propelling', 'calling', 'rolling', 'rolling', 'rolling', 'stargazing', 'sipping', 'feeling', 'packing', 'diving', 'talking', 'jumping']
['cracking', 'taking', 'busting', 'talking', 'pertaining', 'invading', 'hesitating', 'biting', 'biting', 'moving']
['freezing', 'hopping', 'popping', 'selling', 'choking', 'playing', 'playing', 'coming', 'playing', 'spinning', 'sending', 'sending', 'jumping', 'working']
['saying', 'playing', 'beating']
['coming', 'wanting', 'keeping', 'peeling', 'coming']
['flipping', 'gripping', 'dodging', 'walking']
[]
['flowing', 'getting', 'creeping', 'controlling', 'dreaming', 'looking', 'dreaming']
['creeping', 'bouncing', 'feeling']
['hanging', 'sipping', 'vibrating', 'talking', 'thinking', 'fucking', 'hanging', 'sipping', 'vibrating']
['wilding']
['dripping']
['smoking', 'smoking', 'calling', 'smoking']
['sa

In [18]:
# replace slang words ending in "in'" that should be in 'ing'
df1.lyrics = df1.lyrics.apply(lambda x: [replace_slang(word) for word in x.split()])
df1.lyrics = df1.lyrics.apply(lambda x: ' '.join(x))

In [19]:
# words with apostrophes (leftovers)

for i in df1.lyrics.apply(lambda x: [j for j in x.split() if "'" in j]):
    print(i)
    
    print('==================================')

["femalin'", "intellin'", "'", "b's"]
[]
["winter's"]
[]
["signal's", "signal's"]
["shit's", "l's", "heaven's"]
[]
[]
[]
[]
["fiend's", "need's", "angels'", "need's", "need's"]
["hun'", "hun'"]
["vibe's"]
["frenchy's"]
["m's", "m's", "m's", "m's", "m's", "m's"]
["lambo'", "yo'", "yo'"]
["someone's", "shit'll"]


In [20]:
# remove remaining punctuation
df1.lyrics = df1.lyrics.apply(lambda x: re.sub(r'\W+', ' ', x))

In [21]:
# checking for single letters
# REMOVE THESE IN THE STOPWORD REMOVAL STEP

w = set()

for i in df1.lyrics.apply(lambda x: [i for i in x.split() if len(i) < 2]):
    for j in i:
        w.add(j)
        
for i in w:
    print(i in stop_words)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [22]:
# tokenize
df1.lyrics = df1.lyrics.apply(lambda x: word_tokenize(x))

In [23]:
# part-of-speech tagging (for accurate lemmatization)
df1.lyrics = df1.lyrics.apply(lambda tokens: pos_tag(tokens))

In [24]:
test_tag_map = defaultdict(lambda : '---------------------------------> wordnet.NOUN')
test_tag_map['J'] = wordnet.ADJ
test_tag_map['V'] = wordnet.VERB
test_tag_map['R'] = wordnet.ADV

In [25]:
# all tags (stopwords included)

a = set()

for i in df1.lyrics:
    for j in i:
        a.add(j[1])

In [26]:
for i in a:
    print(i, '--->', test_tag_map[i[0]], '--->', i)

JJS ---> a ---> JJS
TO ---> ---------------------------------> wordnet.NOUN ---> TO
WDT ---> ---------------------------------> wordnet.NOUN ---> WDT
MD ---> ---------------------------------> wordnet.NOUN ---> MD
PRP ---> ---------------------------------> wordnet.NOUN ---> PRP
UH ---> ---------------------------------> wordnet.NOUN ---> UH
IN ---> ---------------------------------> wordnet.NOUN ---> IN
EX ---> ---------------------------------> wordnet.NOUN ---> EX
CD ---> ---------------------------------> wordnet.NOUN ---> CD
VB ---> v ---> VB
RP ---> r ---> RP
CC ---> ---------------------------------> wordnet.NOUN ---> CC
JJ ---> a ---> JJ
NNPS ---> ---------------------------------> wordnet.NOUN ---> NNPS
NNS ---> ---------------------------------> wordnet.NOUN ---> NNS
VBP ---> v ---> VBP
WP ---> ---------------------------------> wordnet.NOUN ---> WP
PDT ---> ---------------------------------> wordnet.NOUN ---> PDT
WRB ---> ---------------------------------> wordnet.NOUN ---> 

In [27]:
# tagged stopwords

for i in df1.lyrics.apply(lambda x: x):
    for j in i:
        if j[0] in stop_words:
            print(j)
    print('======================================')

('got', 'VBD')
('me', 'PRP')
('on', 'IN')
('like', 'IN')
('the', 'DT')
('whatever', 'WDT')
('i', 'NN')
('it', 'PRP')
('got', 'VBD')
('me', 'PRP')
('going', 'VBG')
('got', 'VBD')
('me', 'PRP')
('going', 'VBG')
('i', 'NN')
('was', 'VBD')
('as', 'IN')
('out', 'IN')
('in', 'IN')
('the', 'DT')
('then', 'RB')
('a', 'DT')
('came', 'VBD')
('in', 'IN')
('and', 'CC')
('my', 'PRP$')
('up', 'RB')
('to', 'TO')
('the', 'DT')
('down', 'RP')
('on', 'IN')
('my', 'PRP$')
('out', 'IN')
('of', 'IN')
('nowhere', 'RB')
('you', 'PRP')
('came', 'VBD')
('here', 'RB')
('to', 'TO')
('the', 'DT')
('in', 'IN')
('the', 'DT')
('got', 'VBD')
('me', 'PRP')
('got', 'VBD')
('me', 'PRP')
('going', 'VBG')
('they', 'PRP')
('are', 'VBP')
('they', 'PRP')
('we', 'PRP')
('up', 'RP')
('top', 'JJ')
('with', 'IN')
('uh', 'NN')
('the', 'DT')
('with', 'IN')
('got', 'VBD')
('me', 'PRP')
('going', 'VBG')
('on', 'IN')
('we', 'PRP')
('will', 'MD')
('tell', 'VB')
('them', 'PRP')
('we', 'PRP')
('the', 'DT')
('section', 'NN')
('they', 'PR

('yeah', 'RB')
('in', 'IN')
('the', 'DT')
('back', 'NN')
('she', 'PRP')
('said', 'VBD')
('she', 'PRP')
('on', 'IN')
('her', 'PRP$')
('yeah', 'EX')
('are', 'VBP')
('not', 'RB')
('by', 'IN')
('the', 'DT')
('yeah', 'NN')
('this', 'DT')
('how', 'WRB')
('it', 'PRP')
('look', 'VB')
('yeah', 'PRP')
('about', 'IN')
('a', 'DT')
('yeah', 'NN')
('just', 'RB')
('the', 'DT')
('yeah', 'VBP')
('this', 'DT')
('to', 'TO')
('my', 'PRP$')
('i', 'NN')
('am', 'VBP')
('going', 'VBG')
('to', 'TO')
('show', 'VB')
('her', 'PRP$')
('what', 'WP')
('it', 'PRP')
('took', 'VBD')
('got', 'VBD')
('these', 'DT')
('other', 'JJ')
('yeah', 'NN')
('in', 'IN')
('to', 'TO')
('we', 'PRP')
('take', 'VBP')
('it', 'PRP')
('i', 'VBZ')
('am', 'VBP')
('going', 'VBG')
('to', 'TO')
('let', 'VB')
('you', 'PRP')
('know', 'VB')
('the', 'DT')
('move', 'NN')
('i', 'NN')
('am', 'VBP')
('going', 'VBG')
('to', 'TO')
('let', 'VB')
('you', 'PRP')
('know', 'VB')
('the', 'DT')
('move', 'NN')
('i', 'NN')
('am', 'VBP')
('going', 'VBG')
('to', 'TO

('the', 'DT')
('up', 'RP')
('the', 'DT')
('up', 'RP')
('the', 'DT')
('up', 'IN')
('they', 'PRP')
('know', 'VBP')
('me', 'PRP')
('when', 'WRB')
('they', 'PRP')
('see', 'VBP')
('me', 'PRP')
('the', 'DT')
('up', 'RP')
('the', 'DT')
('up', 'RP')
('uh', 'NN')
('yeah', 'NN')
('yeah', 'NN')
('yeah', 'NN')
('like', 'IN')
('we', 'PRP')
('are', 'VBP')
('at', 'IN')
('the', 'DT')
('it', 'PRP')
('is', 'VBZ')
('like', 'IN')
('in', 'IN')
('the', 'DT')
('we', 'PRP')
('just', 'RB')
('i', 'NN')
('gave', 'VBD')
('her', 'PRP$')
('of', 'IN')
('the', 'DT')
('it', 'PRP')
('was', 'VBD')
('to', 'TO')
('the', 'DT')
('did', 'VBD')
('not', 'RB')
('the', 'DT')
('that', 'WDT')
('was', 'VBD')
('out', 'IN')
('of', 'IN')
('afterwards', 'NNS')
('the', 'DT')
('i', 'NN')
('was', 'VBD')
('out', 'IN')
('of', 'IN')
('if', 'IN')
('you', 'PRP')
('take', 'VBP')
('your', 'PRP$')
('out', 'RP')
('do', 'VBP')
('you', 'PRP')
('if', 'IN')
('she', 'PRP')
('take', 'VBP')
('her', 'PRP$')
('out', 'RP')
('do', 'VBP')
('you', 'PRP')
('fir

('me', 'PRP')
('and', 'CC')
('my', 'PRP$')
('i', 'NN')
('we', 'PRP')
('like', 'IN')
('the', 'DT')
('same', 'JJ')
('with', 'IN')
('all', 'DT')
('my', 'PRP$')
('on', 'IN')
('let', 'NN')
('us', 'PRP')
('have', 'VBP')
('with', 'IN')
('the', 'DT')
('you', 'PRP')
('get', 'VBP')
('your', 'PRP$')
('want', 'VBP')
('to', 'TO')
('with', 'IN')
('the', 'DT')
('you', 'PRP')
('got', 'VBD')
('to', 'TO')
('it', 'PRP')
('if', 'IN')
('you', 'PRP')
('going', 'VBG')
('to', 'TO')
('on', 'IN')
('got', 'VBD')
('to', 'TO')
('it', 'PRP')
('in', 'IN')
('the', 'DT')
('i', 'NN')
('can', 'MD')
('not', 'RB')
('it', 'PRP')
('different', 'JJ')
('woah', 'JJ')
('think', 'VBP')
('the', 'DT')
('just', 'RB')
('off', 'RP')
('yeah', 'RB')
('seem', 'VBP')
('like', 'IN')
('the', 'DT')
('i', 'JJ')
('seem', 'VBP')
('like', 'IN')
('the', 'DT')
('i', 'JJ')
('s', 'VB')
('a', 'DT')
('little', 'JJ')
('yeah', 'NN')
('seems', 'VBZ')
('like', 'IN')
('the', 'DT')
('i', 'NN')
('need', 'VBP')
('yeah', 'RB')
('seems', 'VBZ')
('like', 'IN')


('must', 'MD')
('be', 'VB')
('the', 'DT')
('little', 'JJ')
('i', 'VB')
('am', 'VBP')
('all', 'DT')
('in', 'IN')
('this', 'DT')
('i', 'NN')
('old', 'JJ')
('she', 'PRP')
('you', 'PRP')
('know', 'VBP')
('i', 'JJ')
('in', 'IN')
('the', 'DT')
('oh', 'NN')
('yeah', 'NN')
('she', 'PRP')
('i', 'JJ')
('you', 'PRP')
('in', 'IN')
('the', 'DT')
('you', 'PRP')
('know', 'VBP')
('this', 'DT')
('mean', 'NN')
('nothing', 'NN')
('oh', 'RB')
('did', 'VBD')
('not', 'RB')
('i', 'VB')
('your', 'PRP$')
('mhm', 'NN')
('no', 'DT')
('no', 'DT')
('on', 'IN')
('no', 'DT')
('oh', 'IN')
('yeah', 'NN')
('i', 'NN')
('know', 'VBP')
('you', 'PRP')
('me', 'PRP')
('i', 'VBP')
('it', 'PRP')
('are', 'VBP')
('not', 'RB')
('no', 'RB')
('you', 'PRP')
('down', 'RP')
('you', 'PRP')
('i', 'NNS')
('do', 'VBP')
('not', 'RB')
('give', 'VB')
('a', 'DT')
('how', 'WRB')
('it', 'PRP')
('get', 'VB')
('that', 'DT')
('little', 'JJ')
('know', 'VBP')
('i', 'RB')
('this', 'DT')
('uh', 'NN')
('huh', 'VBZ')
('oh', 'RP')
('yeah', 'UH')
('get', 

('and', 'CC')
('as', 'IN')
('your', 'PRP$')
('to', 'TO')
('give', 'VB')
('out', 'RP')
('i', 'NN')
('am', 'VBP')
('just', 'RB')
('thing', 'NN')
('the', 'DT')
('two', 'CD')
('and', 'CC')
('the', 'DT')
('and', 'CC')
('who', 'WP')
('it', 'PRP')
('is', 'VBZ')
('for', 'IN')
('you', 'PRP')
('but', 'CC')
('i', 'VB')
('am', 'VBP')
('back', 'RB')
('on', 'IN')
('the', 'DT')
('anything', 'NN')
('in', 'IN')
('between', 'IN')
('back', 'NN')
('of', 'IN')
('the', 'DT')
('line', 'NN')
('back', 'RB')
('back', 'RB')
('back', 'RB')
('in', 'IN')
('the', 'DT')
('line', 'NN')
('yeah', 'NN')


In [28]:
# tags for non-stopwords

a = set()

for i in df1.lyrics.apply(lambda x: x):
    for j in i:
        if j[0] not in stop_words:
            a.add(j[1])

In [29]:
for i in a:
    print(i, '--->', test_tag_map[i[0]], '--->', i)

NNPS ---> ---------------------------------> wordnet.NOUN ---> NNPS
VBD ---> v ---> VBD
IN ---> ---------------------------------> wordnet.NOUN ---> IN
NN ---> ---------------------------------> wordnet.NOUN ---> NN
JJS ---> a ---> JJS
NNS ---> ---------------------------------> wordnet.NOUN ---> NNS
VBP ---> v ---> VBP
VBN ---> v ---> VBN
RBR ---> r ---> RBR
PDT ---> ---------------------------------> wordnet.NOUN ---> PDT
MD ---> ---------------------------------> wordnet.NOUN ---> MD
VBG ---> v ---> VBG
VB ---> v ---> VB
RB ---> r ---> RB
JJR ---> a ---> JJR
JJ ---> a ---> JJ
VBZ ---> v ---> VBZ
DT ---> ---------------------------------> wordnet.NOUN ---> DT


In [30]:
# tagged non-stopwords

for i in df1.lyrics:
    for j in i:
        if j[0] not in stop_words:
            print(j)
    print('======================================')

('rolling', 'VBG')
('rolling', 'VBG')
('rolling', 'VBG')
('stargazing', 'VBG')
('sipping', 'VBG')
('purp', 'JJ')
('feeling', 'NN')
('barre', 'NN')
('baby', 'NN')
('downed', 'VBD')
('crazy', 'JJ')
('psychedelics', 'NNS')
('crazy', 'JJ')
('hot', 'JJ')
('hell', 'NN')
('heat', 'NN')
('storm', 'NN')
('saved', 'VBD')
('life', 'NN')
('head', 'VBP')
('sky', 'NN')
('knees', 'NNS')
('save', 'VB')
('night', 'NN')
('nighttime', 'JJ')
('rolling', 'VBG')
('rolling', 'VBG')
('rolling', 'VBG')
('stargazing', 'VBG')
('psychedelics', 'NNS')
('crazy', 'JJ')
('niggas', 'NNS')
('femalin', 'VBP')
('excelling', 'VBG')
('intellin', 'VBP')
('propelling', 'VBG')
('ellen', 'JJ')
('kill', 'VB')
('jealous', 'JJ')
('propane', 'JJ')
('repellent', 'NN')
('crazy', 'JJ')
('tour', 'NN')
('brought', 'VBD')
('calling', 'VBG')
('hectic', 'JJ')
('projected', 'VBD')
('cut', 'VBD')
('plug', 'NN')
('interjected', 'JJ')
('rolling', 'VBG')
('rolling', 'VBG')
('rolling', 'VBG')
('stargazing', 'VBG')
('sipping', 'VBG')
('purp', 'J

('screw', 'NN')
('tape', 'NN')
('zone', 'NN')
('man', 'NN')
('pimp', 'JJ')
('phone', 'NN')
('man', 'NN')
('flip', 'NNS')
('platinum', 'NN')
('chrome', 'NNS')
('man', 'NN')
('southside', 'JJ')
('fade', 'NN')
('braids', 'NNS')
('hang', 'VBD')
('shit', 'NN')
('smooth', 'NNS')
('man', 'NN')
('rest', 'NN')
('peace', 'NN')
('screw', 'NN')
('man', 'NN')
('rest', 'NN')
('peace', 'NN')
('screw', 'VB')
('tonight', 'NN')
('slowly', 'RB')
('god', 'NN')
('feel', 'VB')
('love', 'NN')
('drop', 'NN')
('windows', 'NNS')
('screw', 'NN')
('international', 'JJ')
('sound', 'NN')
('south', 'NN')
('shawty', 'NN')
('real', 'JJ')
('action', 'NN')
('pot', 'NN')
('party', 'NN')
('trippers', 'NNS')
('grasshoppers', 'NNS')
('hip', 'NN')
('gathered', 'VBN')
('secrecy', 'NN')
('flying', 'VBG')
('high', 'JJ')
('kite', 'JJ')
('palm', 'NN')
('trees', 'NNS')
('oceans', 'VBP')
('fresh', 'JJ')
('air', 'NN')
('break', 'VB')
('heart', 'NN')
('ride', 'NN')
('night', 'NN')
('visions', 'NNS')
('angles', 'NNS')
('tight', 'VBD')

('eating', 'VBG')
('punane', 'NN')
('bangs', 'NNS')
('wet', 'IN')
('change', 'VB')
('banquet', 'NN')
('solo', 'NN')
('mulsanne', 'NN')
('specs', 'NN')
('pull', 'VB')
('left', 'VBN')
('ice', 'NN')
('froze', 'VBD')
('glow', 'JJ')
('glow', 'JJ')
('glow', 'JJ')
('main', 'JJ')
('bitches', 'NNS')
('thinking', 'VBG')
('main', 'JJ')
('bitch', 'NN')
('hit', 'VBD')
('club', 'NN')
('swap', 'VB')
('hoes', 'NNS')
('gang', 'NN')
('shit', 'VBD')
('switch', 'VB')
('shirt', 'NN')
('stained', 'VBD')
('buy', 'VB')
('visa', 'NN')
('moscow', 'NN')
('fucking', 'VBG')
('travis', 'NN')
('crossed', 'VBN')
('nigga', 'VBZ')
('rich', 'JJ')
('saint', 'NN')
('laurent', 'NN')
('dogs', 'NNS')
('earrings', 'VBZ')
('cost', 'NN')
('quarter', 'NN')
('certified', 'VBN')
('gia', 'NN')
('bae', 'NN')
('smoke', 'NN')
('bitch', 'NN')
('kid', 'NN')
('cudi', 'NN')
('signed', 'VBN')
('ye', 'VB')
('nutted', 'VBN')
('cheek', 'NN')
('nickname', 'NN')
('babyface', 'IN')
('bitch', 'NN')
('swear', 'VBP')
('sex', 'NN')
('fuck', 'NN')
('

('long', 'JJ')
('dreads', 'NNS')
('girl', 'NN')
('network', 'NN')
('net', 'JJ')
('iced', 'VBD')
('watches', 'NNS')
('gang', 'NN')
('patek', 'VBD')
('set', 'NN')
('block', 'NN')
('finesse', 'JJ')
('dropping', 'VBG')
('bombs', 'IN')
('save', 'VB')
('breath', 'NN')
('play', 'VB')
('steel', 'NN')
('tryna', 'VBP')
('funk', 'VB')
('flex', 'JJ')
('dogs', 'NNS')
('ride', 'VB')
('death', 'NN')
('elevator', 'NN')
('hill', 'NN')
('steps', 'NNS')
('freeing', 'VBG')
('man', 'NN')
('forgive', 'VBP')
('forget', 'VB')
('easily', 'RB')
('cut', 'VB')
('phone', 'NN')
('reach', 'VB')
('life', 'NN')
('maze', 'NN')
('phases', 'NNS')
('ventilation', 'NN')
('vacation', 'NN')
('houstonfornication', 'NN')
('mind', 'NN')
('redefine', 'VB')
('renovations', 'NNS')
('space', 'NN')
('coupe', 'NN')
('space', 'NN')
('station', 'NN')
('rise', 'VBP')
('east', 'JJ')
('land', 'NN')
('west', 'NN')
('shit', 'JJ')
('pop', 'NN')
('bust', 'NN')
('check', 'NN')
('plays', 'NNS')
('link', 'VBP')
('squad', 'NN')
('cash', 'NN')
('b

In [31]:
# remove stopwords
df1.lyrics = df1.lyrics.apply(lambda x: [i for i in x if i[0] not in stop_words])

In [32]:
# non-stopwords in each song

for i in df1.lyrics:
    print(' '.join([j[0] for j in i]))
    print('========================')

rolling rolling rolling stargazing sipping purp feeling barre baby downed crazy psychedelics crazy hot hell heat storm saved life head sky knees save night nighttime rolling rolling rolling stargazing psychedelics crazy niggas femalin excelling intellin propelling ellen kill jealous propane repellent crazy tour brought calling hectic projected cut plug interjected rolling rolling rolling stargazing sipping purp feeling barre baby downed crazy psychedelics crazy hot hell heat storm saved life head sky knees save night nighttime crazy days time lay drown thoughts kind traumatized kids tryna alive astroworld relocate told dawgs bring seal faith car notes baby girl played tourist guide keys city rides money problems enemies amenities packing toyota league mosh pit injuries stage diving nosebleeds hit booger sugar nose bleed bounce shit forever knees talking forever week baby mama trophy throwing feel bleed dick jumping feel moby gold beef feel kobe astronomical picked ways feel responsible

In [33]:
wnl = WordNetLemmatizer()

In [34]:
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV

In [35]:
# # lemmatize non-stopwords

# for i in df1.lyrics:
#     for j in i:
#         if j[0] not in stop_words and j[0]:
#             #print(j[0], tag_map[j[1][0]], j[1])
#             print(wnl.lemmatize(j[0], tag_map[j[1][0]]), j[1])
#             print('--------------')
#     print('========================')

# # same done below

In [36]:
# original --> lemmatized --> pos .. ALL (pre-fix)

for i in df1.lyrics:
    for j in i:
        if j[0] not in stop_words:
            print(f'{j[0]:<18} {wnl.lemmatize(j[0], tag_map[j[1][0]]):<18} {j[1]}')
    print('============================================================')

rolling            roll               VBG
rolling            roll               VBG
rolling            roll               VBG
stargazing         stargaze           VBG
sipping            sip                VBG
purp               purp               JJ
feeling            feeling            NN
barre              barre              NN
baby               baby               NN
downed             down               VBD
crazy              crazy              JJ
psychedelics       psychedelics       NNS
crazy              crazy              JJ
hot                hot                JJ
hell               hell               NN
heat               heat               NN
storm              storm              NN
saved              save               VBD
life               life               NN
head               head               VBP
sky                sky                NN
knees              knee               NNS
save               save               VB
night              night              NN
nightt

town               town               NN
cooked             cook               VBN
wake               wake               VB
feel               feel               VBP
dreaming           dream              VBG
sunday             sunday             NN
willie             willie             JJ
beamen             beamen             NNS
mondays            monday             NNS
weekend            weekend            NN
coco               coco               NN
ice                ice                NN
bend               bend               VBP
piping             piping             NN
bust               bust               VBD
cloud              cloud              NN
shoot              shoot              NN
lightning          lightning          NN
pop                pop                NN
sleep              sleep              VB
wake               wake               VB
spread             spread             VBP
sheets             sheet              NNS
pussy              pussy              NNS
good  

boss               bos                NN
man                man                NN
cash               cash               NN
desk               desk               NN
coupe              coupe              NN
fast               fast               RB
flame              flame              NN
island             island             NN
cash               cash               NN
gunna              gunna              NN
hopped             hop                VBD
learjet            learjet            NN
pradas             pradas             JJ
color              color              NN
ccs                cc                 NN
kick               kick               VB
cup                cup                NN
codeine            codeine            NN
pointers           pointer            NNS
eliantte           eliantte           RB
dripping           drip               VBG
team               team               NN
wet                wet                NN
mad                mad                JJ
smoking      

main               main               JJ
sweet              sweet              NN
candy              candy              NN
cane               cane               NN
drop               drop               NN
pop                pop                NN
bang               bang               VB
life               life               NN
change             change             VB
hidden             hidden             JJ
hills              hill               NNS
deep               deep               VBP
main               main               JJ
sweet              sweet              NN
candy              candy              NN
cane               cane               NN
drop               drop               NN
pop                pop                NN
bang               bang               VB
yah                yah                NN
ventilation        ventilation        NN
vacation           vacation           NN
houstonfornication houstonfornication NN
mind               mind               NN
redefine      

In [37]:
# original --> lemmatized --> pos .. PLURALS (pre-fix)

for i in df1.lyrics:
    for j in i:
        if j[0] not in stop_words and j[1] == 'NNS':
            print(f'{j[0]:<18} {wnl.lemmatize(j[0], tag_map[j[1][0]]):<18} {j[1]}')
    print('============================================================')

psychedelics       psychedelics       NNS
knees              knee               NNS
psychedelics       psychedelics       NNS
niggas             nigga              NNS
psychedelics       psychedelics       NNS
knees              knee               NNS
days               day                NNS
thoughts           thought            NNS
kids               kid                NNS
rides              ride               NNS
problems           problem            NNS
enemies            enemy              NNS
amenities          amenity            NNS
injuries           injury             NNS
nosebleeds         nosebleed          NNS
knees              knee               NNS
ways               way                NNS
problems           problem            NNS
days               day                NNS
days               day                NNS
lights             light              NNS
demons             demon              NNS
doors              door               NNS
fades              fade           

In [38]:
# lemmatize words that end with 'ss' (before fixes)

for i in df1.lyrics:
    for j in i:
        if j[0] not in stop_words and j[0].endswith('ss'):
            print(j)
            print(wnl.lemmatize(j[0], tag_map[j[1][0]]), j[1])
            print(wordnet.morphy(j[0], tag_map[j[1][0]]), j[1])
            print('--------------------------')
    print('========================')

('pass', 'NN')
pas NN
pass NN
--------------------------
('pass', 'VB')
pass VB
pass VB
--------------------------
('pass', 'VB')
pass VB
pass VB
--------------------------
('pass', 'VBP')
pass VBP
pass VBP
--------------------------
('guess', 'NN')
guess NN
guess NN
--------------------------
('harass', 'NN')
harass NN
None NN
--------------------------
('ass', 'NN')
as NN
ass NN
--------------------------
('pass', 'NN')
pas NN
pass NN
--------------------------
('kiss', 'VB')
kiss VB
kiss VB
--------------------------
('kiss', 'VB')
kiss VB
kiss VB
--------------------------
('flawless', 'NN')
flawless NN
None NN
--------------------------
('dress', 'NN')
dress NN
dress NN
--------------------------
('mess', 'NN')
mess NN
mess NN
--------------------------
('vvss', 'VBP')
vvss VBP
None VBP
--------------------------
('boss', 'NN')
bos NN
boss NN
--------------------------
('flawless', 'NN')
flawless NN
None NN
--------------------------
('dress', 'NN')
dress NN
dress NN
-------------

In [39]:
def lemmatize_ss(word_tag_pair):
    '''lemmatize with a focus on words ending in ss'''
    
    word = word_tag_pair[0]
    tag = tag_map[word_tag_pair[1][0]]
    
    if word.endswith('ss'):
        lemma = wordnet.morphy(word, tag)
        
        if lemma:
            return lemma
        return wnl.lemmatize(word, tag)
    else:
        return wnl.lemmatize(word, tag)

In [40]:
lemmatize_ss(('pass', 'NN'))

'pass'

In [41]:
lemmatize_ss(('harass', 'NN'))

'harass'

In [42]:
lemmatize_ss(('congress', 'JJ'))

'congress'

In [43]:
lemmatize_ss(('boss', 'NN'))

'boss'

In [44]:
# lemmatize words that end with 'ss' (fixed)

for i in df1.lyrics:
    for j in i:
        if j[0] not in stop_words and j[0].endswith('ss'):
            print(j)
            print(lemmatize_ss(j), j[1])
            print('--------------------------')
    print('========================')

('pass', 'NN')
pass NN
--------------------------
('pass', 'VB')
pass VB
--------------------------
('pass', 'VB')
pass VB
--------------------------
('pass', 'VBP')
pass VBP
--------------------------
('guess', 'NN')
guess NN
--------------------------
('harass', 'NN')
harass NN
--------------------------
('ass', 'NN')
ass NN
--------------------------
('pass', 'NN')
pass NN
--------------------------
('kiss', 'VB')
kiss VB
--------------------------
('kiss', 'VB')
kiss VB
--------------------------
('flawless', 'NN')
flawless NN
--------------------------
('dress', 'NN')
dress NN
--------------------------
('mess', 'NN')
mess NN
--------------------------
('vvss', 'VBP')
vvss VBP
--------------------------
('boss', 'NN')
boss NN
--------------------------
('flawless', 'NN')
flawless NN
--------------------------
('dress', 'NN')
dress NN
--------------------------
('mess', 'NN')
mess NN
--------------------------
('vvss', 'VBP')
vvss VBP
--------------------------
('boss', 'NN')
boss 

In [45]:
# original --> lemmatized --> pos .. ALL (FIXED ss)

for i in df1.lyrics:
    for j in i:
        if j[0] not in stop_words:
            print(f'{j[0]:<18} {lemmatize_ss(j):<18} {j[1]}')
    print('============================================================')

rolling            roll               VBG
rolling            roll               VBG
rolling            roll               VBG
stargazing         stargaze           VBG
sipping            sip                VBG
purp               purp               JJ
feeling            feeling            NN
barre              barre              NN
baby               baby               NN
downed             down               VBD
crazy              crazy              JJ
psychedelics       psychedelics       NNS
crazy              crazy              JJ
hot                hot                JJ
hell               hell               NN
heat               heat               NN
storm              storm              NN
saved              save               VBD
life               life               NN
head               head               VBP
sky                sky                NN
knees              knee               NNS
save               save               VB
night              night              NN
nightt

bicentennial       bicentennial       JJ
man                man                NN
city               city               NN
slam               slam               NN
trippy             trippy             JJ
xans               xans               NNPS
lost               lose               VBD
grams              gram               NNS
cam                cam                NN
video              video              NN
dance              dance              NN
rules              rule               NNS
pick               pick               VB
choose             choose             VB
set                set                VB
picks              pick               NNS
shoot              shoot              NN
chop               chop               NN
screwed            screw              VBN
told               tell               VBD
buy                buy                VB
boobs              boob               NN
god                god                NN
box                box                NN
opps    

wake               wake               VB
spread             spread             VBP
sheets             sheet              NNS
pussy              pussy              NNS
good               good               JJ
pussy              pussy              NNS
sweet              sweet              JJ
wake               wake               VB
flowing            flow               VBG
streams            stream             NNS
hot                hot                JJ
feet               foot               NNS
wake               wake               VB
wake               wake               VB
feel               feel               VBP
creeping           creep              VBG
controlling        control            VBG
moving             move               VBG
lucid              lucid              JJ
dreaming           dream              VBG
seasons            season             NNS
beltway            beltway            NN
hood               hood               NNS
crowd              crowd              NN
lo

shit               shit               VBP
alive              alive              JJ
revive             revive             VB
vibe               vibe               NN
collide            collide            VB
guys               guy                NNS
hills              hill               NNS
ghetto             ghetto             VB
ties               tie                NNS
talking            talk               VBG
frenchy            frenchy            JJ
thinking           think              VBG
ocean              ocean              JJ
prime              prime              NN
open               open               JJ
mouth              mouth              NN
open               open               JJ
mind               mind               NN
open               open               VB
door               door               NN
open               open               JJ
stress             stress             JJ
thighs             thighs             JJ
jet                jet                NN
quest     

fly                fly                VBP
broads             broad              NNS
fly                fly                VBP
dawgs              dawgs              NN
atlanta            atlanta            VB
cut                cut                NN
medusa             medusa             NNS
lay                lay                VBD
low                low                JJ
roll               roll               VB
calm               calm               VB
moving             move               VBG
high               high               JJ
speed              speed              NN
send               send               VBP
text               text               VB
moving             move               VBG
moving             move               VBG
time               time               NN
flexing            flex               VBG
flexing            flex               VBG
exercise           exercise           VB
exercise           exercise           NN
exercise           exercise           NN
exerc

In [46]:
# original --> lemmatized --> pos .. PLURALS (FIXED(?))

for i in df1.lyrics:
    for j in i:
        if j[0] not in stop_words and j[1] == 'NNS':
            print(f'{j[0]:<18} {lemmatize_ss(j):<18} {j[1]}')
    print('============================================================')

psychedelics       psychedelics       NNS
knees              knee               NNS
psychedelics       psychedelics       NNS
niggas             nigga              NNS
psychedelics       psychedelics       NNS
knees              knee               NNS
days               day                NNS
thoughts           thought            NNS
kids               kid                NNS
rides              ride               NNS
problems           problem            NNS
enemies            enemy              NNS
amenities          amenity            NNS
injuries           injury             NNS
nosebleeds         nosebleed          NNS
knees              knee               NNS
ways               way                NNS
problems           problem            NNS
days               day                NNS
days               day                NNS
lights             light              NNS
demons             demon              NNS
doors              door               NNS
fades              fade           

###### morphy may have been able to fix (or worsen) other non-stopwords.. I WOULD HAVE TO COME BACK AND CHECK .. try morphy on all non-stopwords, not just those ending with 'ss'

In [47]:
# lemmatize
df1.lyrics = df1.lyrics.apply(lambda x: [lemmatize_ss(pair) for pair in x])
df1.lyrics = df1.lyrics.apply(lambda x: ' '.join(x))

In [48]:
df1.lyrics

STARGAZING               roll roll roll stargaze sip purp feeling barre...
CAROUSEL                 crack boy big tuck freight train tuck astrowor...
SICKO MODE               astro sun freeze cold winter dawg louis belt p...
R.I.P. SCREW             rest peace screw tonight slowly cruise screw f...
STOP TRYING TO BE GOD    real action pot party tripper grasshopper hip ...
NO BYSTANDERS            party end motel lay sin tryna revenge love spe...
SKELETONS                feel slow motion float speed louder high high ...
WAKE UP                  wake spread sheet pussy good pussy sweet wake ...
5% TINT                  creep window endzone enzo love smoke love liq ...
NC-17                    bitch swear sex fuck chain chain sex hang gang...
ASTROTHUNDER             high life life fiend life fiend distant life l...
YOSEMITE                 ice neck flawless baguettes hop jet barely res...
CAN'T SAY                mad smoke hella weed alcohol shawty lick clean...
WHO? WHAT!               

In [49]:
df1

,url,lyrics
STARGAZING,https://genius.com/Travis-scott-stargazing-lyrics,roll roll roll stargaze sip purp feeling barre...
CAROUSEL,https://genius.com/Travis-scott-carousel-lyrics,crack boy big tuck freight train tuck astrowor...
SICKO MODE,https://genius.com/Travis-scott-sicko-mode-lyrics,astro sun freeze cold winter dawg louis belt p...
R.I.P. SCREW,https://genius.com/Travis-scott-rip-screw-lyrics,rest peace screw tonight slowly cruise screw f...
STOP TRYING TO BE GOD,https://genius.com/Travis-scott-stop-trying-to...,real action pot party tripper grasshopper hip ...
NO BYSTANDERS,https://genius.com/Travis-scott-no-bystanders-...,party end motel lay sin tryna revenge love spe...
SKELETONS,https://genius.com/Travis-scott-skeletons-lyrics,feel slow motion float speed louder high high ...
WAKE UP,https://genius.com/Travis-scott-wake-up-lyrics,wake spread sheet pussy good pussy sweet wake ...
5% TINT,https://genius.com/Travis-scott-5-tint-lyrics,creep window endzone enzo love smoke love liq ...
NC-17,https://genius.com/Travis-scott-nc-17-lyrics,bitch swear sex fuck chain chain sex hang gang...


In [50]:
import pickle

with open('astroworld.pkl', 'wb') as f:
    pickle.dump(df1, f)

###### words like 'psychedelics' will only transformed to the root/base form with stemming

wordnet.morphy('psychedelics', 'n')

###### function from Codecademy.. NAIVE, CONTEXTLESS

def get_pos(word):
    from collections import Counter

    synsets = wordnet.synsets(word)
    pos_counts = Counter()
    
    pos_counts['n'] = len([item for item in synsets if item.pos()=='n'])
    pos_counts['v'] = len([item for item in synsets if item.pos()=='v'])
    pos_counts['a'] = len([item for item in synsets if item.pos()=='a'])
    pos_counts['r'] = len([item for item in synsets if item.pos()=='r'])
    
    most_likely_pos = pos_counts.most_common(1)[0][0]
    
    return most_likely_pos

for i in df2.lyrics:
    for k in [(j, get_pos(j)) for j in i if j not in stop_words]:
        print(k)
    break

for i in df2.lyrics:
    print([tuple([j, get_pos(j)]) for j in i])
    print('========================')

from nltk.parse.corenlp import CoreNLPParser as CoreNLPParser1
from nltk.parse import CoreNLPParser as CoreNLPParser2

CoreNLPParser1 == CoreNLPParser2

parser = CoreNLPParser1()

list(parser.tokenize(df2.lyrics[0]))

pos_tagger = CoreNLPParser1(tagtype='pos')

stargazing = list(pos_tagger.tag(word_tokenize(df2.lyrics[0])))

stargazing

from nltk.tag.stanford import StanfordPOSTagger

    spos = StanfordPOSTagger(
        #model_filename = '/Users/meekaymacbook/stanford-postagger-2018-10-16/models/english-bidirectional-distsim.tagger',
        model_filename = '/Users/meekaymacbook/stanford-postagger-2018-10-16/models/english-left3words-distsim.tagger',
        path_to_jar = '/Users/meekaymacbook/stanford-postagger-2018-10-16/stanford-postagger.jar',
        encoding = 'utf8'
    )

spos.tag(word_tokenize(df2.lyrics[0]))

from nltk.tag.senna import SennaTagger

senna_tagger = SennaTagger()

from nltk.util import bigrams, trigrams

tuple(bigrams(df2.lyrics[0].split()))

tuple(trigrams(df2.lyrics[0].split()))

from nltk.corpus import brown
from nltk import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

train_sents = brown.tagged_sents(categories='news')

unigram_tagger = UnigramTagger(train_sents)

unigram_tagger.tag(word_tokenize(df2.lyrics[0]))

unigram_tagger.evaluate(train_sents)

bigram_tagger = BigramTagger(train_sents)

bigram_tagger.tag(word_tokenize(df2.lyrics[0]))

t0 = DefaultTagger('---')
t1 = UnigramTagger(train_sents, backoff=t0)
t2 = BigramTagger(train_sents, backoff=t1)
t3 = TrigramTagger(train_sents, backoff=t2)

t3.tag(word_tokenize(df2.lyrics[0]))

t0 = DefaultTagger('---')
t1 = UnigramTagger(train_sents, backoff=t0)
t2 = BigramTagger(train_sents, backoff=t1)
t3 = TrigramTagger(train_sents, backoff=t2)

t3.tag(word_tokenize(df2.lyrics[0]))

###### assuming there are no contractions surrounded by hyphens

###### tag before removing stopwords to preserve context

###### tag with regex tagger for preprocessing? for things like non-gerunds with 'in' (ing slang)'

###### 'in' test cases

a = ['robitussing', 'nothing', 'intelling', 'drawing']

rfunky_word(a)

funky_word(a)

wordnet.synsets(a)

get_pos(a)

import time

start = time.time()
print(replace_slang('fundin'))
end = time.time()
end-start

b = ['blessing', 'nothing', 'funding', 'wiring', 'jumping']

lemmatize_noun(b)

lemmatize_verb(b)

get_pos(b)

tuple(zip(wordnet.synsets(b, get_pos(b)), [i.definition() for i in wordnet.synsets(b, get_pos(b))]))

tuple(zip(wordnet.synsets(b, get_pos('n')), [i.definition() for i in wordnet.synsets(b, get_pos('n'))]))

tuple(zip(wordnet.synsets(b, get_pos('v')), [i.definition() for i in wordnet.synsets(b, get_pos('v'))]))

### Funky words (adverbs, non-gerunds ending in -ing)

In [51]:
def funky_word(word): #tester
    synsets = wordnet.synsets(word)
    
    if len(synsets) == 0:
        return word
    
    for i in wordnet.synsets(word):
        print(i)

        if i.lemmas():
            print('lemmas:', i.lemma_names(), '\n')

            for j in i.lemmas():
                print('LEMMA-HERE: ------->', j.name())
                print('pos:', i.pos())

                if j.pertainyms():
                    print('pertainyms:', j.pertainyms(), '\n')
                    for k in j.pertainyms():
                        print(k.name())
                    # print('yes pertainym')
                    # pertainym function?
                else:
                    print('no pertainym, move to definitions...')

                    # need a definition function?
                    # if no definitions, leave word as is, else STEM IT
                    if i.definition():
                        print('DEFINITION:', i.definition(), '\n')
                        stemmed = stem(word)

                        # check if stem is a real word
                        if len(wordnet.synsets(stemmed)) > 0:
                            print('accepted stem:', stemmed)
                        else:
                            print('stemmed form is invalid.. sticking with OG')
                        # if stem is a real word, return it, else leave original
                    else:
                        print('no definition..')
                        # return original?
                print('----') # separate lemmas
        else:
            print('no lemmas.. gotta check for definitions')
            # check if definition
            print(i.definition())
        print('====================')

In [52]:
def rfunky_word(word):
    synsets = wordnet.synsets(word)
    
    ## necessary?
    if not synsets:
        return word
        ##return None
        
    ## necessary?
    target = word + '-->' + 'NOTHING FOUND'

    for i in wordnet.synsets(word):
        if i.lemmas():
            for j in i.lemmas():
                if j.pertainyms():
                    for k in j.pertainyms():
                        if stem(k.name()) == stem(word) or stem(k.name()) in word:
                            #return word + '-->' + 'NOTHING FOUND'
                            return k.name()
                        else:
                            print('different from OG word, but may still work')
                else:
                    if i.definition():
                        stemmed = stem(word)

                        if wordnet.synsets(stemmed):
                            target = stemmed + f'---stemmed from {word}--->' + str(j)
                        else:
                            target = word + '-->' + str(j)
                    else:
                        target = word
        else:
#             if i.definition():
#                 stemmed = stem(word)

#                 if len(wordnet.synsets(stemmed)) > 0:
#                     return stemmed
#                 else:
#                     return word

            print('no lemmas.. gotta check for definitions')
            ## check if definition
            #print(i.definition())
            
    return target

rfunky_word('filling')

funky_word('filling')

lemmatize_noun('filling')

lemmatize_verb('filling')

rfunky_word('terribly')

funky_word('terribly')

rfunky_word('brightly')

funky_word('brightly')

rfunky_word('wilding')

funky_word('wilding')

rfunky_word('nothing') # goes to very end

funky_word('nothing')

###### for words like 'wilding' where there is no shortened lemma (i.e. lemma same as original), stem it if a definition exists

### ADVERB SHIT

wnl.lemmatize('brightly', 'r')

get_pos('brightly')

funky_word('brightly')

funky_word('brightli')

###### misc

c = 'psychedelics' # try to singularize
c = 'psychedelic'

get_pos(c)

funky_word(c)

    # from collections import defaultdict

    # tag_map = defaultdict(lambda : wn.NOUN)
    # tag_map['J'] = wn.ADJ
    # tag_map['V'] = wn.VERB
    # tag_map['R'] = wn.ADV

    # for word, tag in pos_tag(df2.lyrics[0]):
    #     if word.endswith('s'):
    #         lemma = wnl.lemmatize(word, tag_map[tag[0]])

    #         if lemma.endswith('s'):
    #             print(tag, word, '----->', lemma)

###### messing with TextBlob

In [53]:
from textblob import TextBlob as tb

In [54]:
blob = tb('pray')

In [55]:
blob.pos_tags

[('pray', 'NN')]

blob = tb('psychedelics')
blob.words[0].singularize()

blob = tb(df2.lyrics[2])

for word in blob.words:
    if word[-1] == 's':
        print(word)
        print(word.singularize())
        print()

blob.noun_phrases

blob.tags

blob.pos_tags

[tag[1] for tag in blob.pos_tags]

from collections import Counter

c = Counter()

for i in df2.lyrics:
    c.update(i)

c

from nltk.corpus.reader.wordnet import Synset as ss

[i for i in dir(ss) if not i.startswith('_')]